In [1]:
import pickle
import torch
from torch_geometric.data import Data,DataLoader
from functions_refactor import *
from pytorch_util import *
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# fixed parameters
hyper_epoch = 25
threshold = -1.3
epochs = 50
batch_size = 32
clip = 2
lr = 1e-4
epochs_type = epochs

In [ ]:
#for _ in range(hyper_epoch):
    # model parameters

#     head = np.random.choice([feedforwardHead_Update,feedforwardCombineHead_Update],p=[0.6,0.4])
#     data = np.random.choice(['../Data/{}_data_ACSF_expand_PCA_otherInfo.pickle',\
#                              '../Data/{}_data_SOAP_expand_PCA_otherInfo.pickle',\
#                              '../Data/{}_data_atomInfo_otherInfo.pickle',\
#                              '../Data/{}_data_ACSF_SOAP_atomInfo_otherInfo.pickle'
#                             ])
#     logLoss = np.random.choice([True,False])
#     weight = np.random.rand()*3 if logLoss else np.random.rand()*0.8
#     dim = int(np.random.choice([256,512]))#
#     interleave = np.random.choice([True,False])
#     BatchNorm = np.random.choice([True,False],p=[0.6,0.4])
#     if interleave:
#         if dim == 768:
#             layer1 = int(np.random.choice([3,4]))
#             layer2 = layer1
#         else:
#             layer1 = int(np.random.choice([3,4,5]))
#             layer2 = layer1            
#     else:
#         if dim == 768:        
#             layer1 = int(np.random.choice([3,4]))
#             layer2 = int(np.random.choice([3,4]))
#         else:
#             layer1 = int(np.random.choice([3,4,5]))
#             layer2 = int(np.random.choice([3,4,5]))
            
#     if dim == 768:
#         factor = int(np.random.choice([2,3]))
#     else:
#         factor = int(np.random.choice([2,3,4]))
        
#     useMax = np.random.choice([True,False])
head = feedforwardHead_Update
data = '../Data/{}_data_ACSF_expand_PCA_otherInfo.pickle'
dim = 512
logLoss = False
weight = 0.6288604599490546
layer1,layer2,factor = 3,3,2
BatchNorm = True
useMax = False
interleave = False
print('\ntraining on {}\n'.format('_'.join([str(i).split('}')[1] if '}' in str(i) else str(i) \
                                    for i in [head,data,dim,logLoss,weight,layer1,layer2,factor,\
                                              BatchNorm,useMax,interleave]])))

train_dl,val_dl = get_data(data,batch_size)
model = GNN_MataLayer(head,head_mol2,head_atom,head_edge,\
                      dim,layer1,layer2,factor,**data_dict[data],\
                      BatchNorm=BatchNorm,useMax=useMax,interleave=interleave).to('cuda:0')
paras = trainable_parameter(model)
opt = Adam(paras,lr=lr)
scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5)

model,train_loss_perType,val_loss_perType,bestWeight = train_type_earlyStop(opt,model,epochs_type,train_dl,val_dl,paras,clip,\
                                                                scheduler=scheduler,logLoss=logLoss,weight=weight,threshold=threshold)
#     if model is not None:
#         save_results2(train_loss_perType,val_loss_perType,head,data,dim,logLoss,weight,layer1,layer2,factor,\
#                         BatchNorm,useMax,interleave)


training on <class 'functions_refactor.feedforwardHead_Update'>__data_ACSF_expand_PCA_otherInfo.pickle_512_False_0.6288604599490546_3_3_2_True_False_False

epoch:0, train_loss: +2.122, val_loss: -0.506, 
train_vector: +1.11|+0.64|+0.09|-0.39|-0.41|+0.11|-0.43|-0.80, 
val_vector  : +0.32|+0.19|-0.30|-0.98|-0.88|-0.27|-0.94|-1.18



In [4]:
def train_type_earlyStop(opt,model,epochs,train_dl,val_dl,paras,clip,typeTrain=False,\
                         train_loss_list=None,val_loss_list=None,scheduler=None,logLoss=True,weight=None,threshold=0):
    # add early stop and weight for hyper Search
    # add early stop if nan for hyper3
    since = time.time()
    
    lossBest = [1e6] * 8
    bestWeight = [None] * 8
    if train_loss_list is None:
        train_loss_list,val_loss_list = [],[]
        epoch0 = 0
    else:
        epoch0 = len(train_loss_list)
        
    opt.zero_grad()
    for epoch in range(epochs):
        # training #
        model.train()
        np.random.seed()
        train_loss = 0
        train_loss_perType = np.zeros(8)
        val_loss = 0
        val_loss_perType = np.zeros(8)
        
        for i,data in enumerate(train_dl):
            data = data.to('cuda:0')
            loss,loss_perType = model(data,True,typeTrain,logLoss,weight)
            # check nan
            if np.isnan(loss.item()):
                print('-----stop due to nan-----')
                #model.load_state_dict(WeightB4)
                return data,model,None,None            
            loss.backward()
            clip_grad_value_(paras,clip)
            opt.step()
            opt.zero_grad()
            #WeightB4 = copy.deepcopy(model.state_dict())
            train_loss += loss.item()
            train_loss_perType += loss_perType.cpu().detach().numpy()

                    
        # evaluating #
        model.eval()
        with torch.no_grad():
            for j,data in enumerate(val_dl):
                data = data.to('cuda:0')
                loss,loss_perType = model(data,True,typeTrain,True,None)
                val_loss += loss.item()
                val_loss_perType += loss_perType.cpu().detach().numpy()
        
        # early stop
        val_loss = val_loss/j
        if (epoch==20) and (val_loss>threshold):
            print('-----stop due to poor performance-----')
            return None,None,None,None
        

        # save model
        val_loss_perType = val_loss_perType/j
        for index_ in range(8):
            if val_loss_perType[index_]<lossBest[index_]:
                lossBest[index_] = val_loss_perType[index_]
                bestWeight[index_] = copy.deepcopy(model.state_dict())
            
        print('epoch:{}, train_loss: {:+.3f}, val_loss: {:+.3f}, \ntrain_vector: {}, \nval_vector  : {}\n'.format(epoch+epoch0,train_loss/i,val_loss,\
                                                            '|'.join(['%+.2f'%i for i in train_loss_perType/i]),\
                                                            '|'.join(['%+.2f'%i for i in val_loss_perType])))
        train_loss_list.append(train_loss_perType/i)
        val_loss_list.append(val_loss_perType)
        if scheduler is not None:
            scheduler.step(val_loss)
            
    time_elapsed = time.time() - since
    print('Training completed in {}s'.format(time_elapsed))
    return model,train_loss_list,val_loss_list,bestWeight

In [3]:
    head = feedforwardHead_Update
    data = '../Data/{}_data_ACSF_expand_PCA_otherInfo.pickle'
    dim = 512
    logLoss = False
    weight = 0.6288604599490546
    layer1,layer2,factor = 4,3,4
    BatchNorm = False
    useMax = False
    interleave = False
    print('\ntraining on {}\n'.format('_'.join([str(i).split('}')[1] if '}' in str(i) else str(i) \
                                        for i in [head,data,dim,logLoss,weight,layer1,layer2,factor,\
                                                  BatchNorm,useMax,interleave]])))

    train_dl,val_dl = get_data(data,batch_size)


training on <class 'functions_refactor.feedforwardHead_Update'>__data_ACSF_expand_PCA_otherInfo.pickle_512_False_0.6288604599490546_4_3_4_False_False_False



In [6]:
forward  = []
def hook_forward(m, i, o):
    if isinstance(o,tuple):
        forward.append([m]+[(item.max(),item.min(),torch.isnan(item).sum()) for item in o])
    else:
        forward.append([m]+[o.max(),o.min(),torch.isnan(o).sum()])

backward  = []
def hook_backward(m, i, o):
    if isinstance(o,tuple):
        backward.append([m]+[(item.max(),item.min(),torch.isnan(item).sum()) for item in o])
    else:
        backward.append([m]+[o.max(),o.min(),torch.isnan(o).sum()])

In [4]:
from torch.nn import Sequential,Linear,ReLU,BatchNorm1d
class head_mol3(torch.nn.Module):
    def __init__(self,dim,mol_shape):
        factor = 2
        super(head_mol3, self).__init__()
        self.linear = Sequential(BatchNorm1d(dim),Linear(dim, dim*factor),ReLU(), \
                                 BatchNorm1d(dim*factor),Linear(dim*factor, mol_shape))
        
    def forward(self,u):
        out = self.linear(u).squeeze(1)
        return out
    
class head_atom3(torch.nn.Module):
    def __init__(self,dim,atom_shape):
        factor = 2
        super(head_atom3, self).__init__()
        self.linear = Sequential(BatchNorm1d(dim),Linear(dim, dim*factor),ReLU(), \
                                 BatchNorm1d(dim*factor),Linear(dim*factor, atom_shape))
        
    def forward(self,out):
        out = self.linear(out).squeeze(1)
        return out

class head_edge3(torch.nn.Module):
    def __init__(self,dim,edge_shape):
        factor = 2
        super(head_edge3, self).__init__()
        self.linear = Sequential(BatchNorm1d(dim),Linear(dim, dim*factor),ReLU(), \
                                 BatchNorm1d(dim*factor),Linear(dim*factor, edge_shape))
        
    def forward(self,out):
        out = self.linear(out).squeeze(1)
        return out

In [5]:
    model = GNN_MataLayer(head,head_mol3,head_atom3,head_edge3,\
                          dim,layer1,layer2,factor,**data_dict[data],\
                          BatchNorm=BatchNorm,useMax=useMax,interleave=interleave).to('cuda:0')
    paras = trainable_parameter(model)
    opt = Adam(paras,lr=lr)
    scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5)
#     for name, layer in model._modules.items():
#         layer.register_forward_hook(hook_forward)
#         layer.register_backward_hook(hook_backward)
    data,model,_,_ = train_type_earlyStop(opt,model,epochs_type,train_dl,val_dl,paras,clip,\
                                                                    scheduler=scheduler,logLoss=logLoss,weight=weight,threshold=threshold)

epoch:0, train_loss: +2.245, val_loss: -0.494, 
train_vector: +1.30|+0.78|+0.11|-0.26|-0.39|+0.12|-0.35|-0.78, 
val_vector  : +0.52|+0.04|-0.38|-0.95|-0.91|-0.24|-0.88|-1.17



RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 15.75 GiB total capacity; 14.54 GiB already allocated; 8.06 MiB free; 61.67 MiB cached)

In [9]:
def inspect_data(data):
    print(data.max(),data.min(),torch.isnan(data).sum())

In [10]:
attr_list = ['batch','edge_attr','edge_attr3','edge_attr4','edge_index','edge_index3','x','y','y_atom','y_coupling','y_mol']

In [11]:
for attr in attr_list:
    print(attr)
    inspect_data(getattr(data,attr))

batch
tensor(31, device='cuda:0') tensor(0, device='cuda:0') tensor(0, device='cuda:0')
edge_attr
tensor(1.6122, device='cuda:0') tensor(0., device='cuda:0') tensor(0, device='cuda:0')
edge_attr3
tensor(1., device='cuda:0') tensor(0., device='cuda:0') tensor(0, device='cuda:0')
edge_attr4
tensor(3.7523, device='cuda:0') tensor(0., device='cuda:0') tensor(0, device='cuda:0')
edge_index
tensor(575, device='cuda:0') tensor(0, device='cuda:0') tensor(0, device='cuda:0')
edge_index3
tensor(575, device='cuda:0') tensor(1, device='cuda:0') tensor(0, device='cuda:0')
x
tensor(12.9760, device='cuda:0') tensor(-20.6225, device='cuda:0') tensor(0, device='cuda:0')
y
tensor(199.0140, device='cuda:0') tensor(-18.0921, device='cuda:0') tensor(0, device='cuda:0')
y_atom
tensor(13.4248, device='cuda:0') tensor(-11.1703, device='cuda:0') tensor(0, device='cuda:0')
y_coupling
tensor(6.5124, device='cuda:0') tensor(-5.2078, device='cuda:0') tensor(0, device='cuda:0')
y_mol
tensor(2.6854, device='cuda:0')

In [11]:
data

Batch(batch=[545], edge_attr=[1118, 44], edge_attr3=[1486, 8], edge_attr4=[1486, 26], edge_index=[2, 1118], edge_index3=[2, 1486], x=[545, 32], y=[1486], y_atom=[545, 10], y_coupling=[1486, 4], y_mol=[32, 4])

In [12]:
forward

[[Sequential(
    (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=32, out_features=2048, bias=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=2048, out_features=512, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
  ),
  tensor(6.1014, device='cuda:0', grad_fn=<MaxBackward1>),
  tensor(-0.0543, device='cuda:0', grad_fn=<MinBackward1>),
  tensor(0, device='cuda:0')],
 [Sequential(
    (0): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=34, out_features=2048, bias=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=2048, out_features=512, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
  ),
  tensor(6.2015, device='cuda:0', grad_fn=

In [13]:
[torch.isnan(w).sum() for w in model.parameters()]

[tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, dev

In [14]:
[(n,w.max(),w.min()) for n,w in model.named_parameters()]

[('lin_node.0.weight',
  tensor(0.9584, device='cuda:0', grad_fn=<MaxBackward1>),
  tensor(0.0001, device='cuda:0', grad_fn=<MinBackward1>)),
 ('lin_node.0.bias',
  tensor(0., device='cuda:0', grad_fn=<MaxBackward1>),
  tensor(0., device='cuda:0', grad_fn=<MinBackward1>)),
 ('lin_node.1.weight',
  tensor(0.1768, device='cuda:0', grad_fn=<MaxBackward1>),
  tensor(-0.1768, device='cuda:0', grad_fn=<MinBackward1>)),
 ('lin_node.1.bias',
  tensor(0.1767, device='cuda:0', grad_fn=<MaxBackward1>),
  tensor(-0.1768, device='cuda:0', grad_fn=<MinBackward1>)),
 ('lin_node.3.weight',
  tensor(0.9991, device='cuda:0', grad_fn=<MaxBackward1>),
  tensor(0.0009, device='cuda:0', grad_fn=<MinBackward1>)),
 ('lin_node.3.bias',
  tensor(0., device='cuda:0', grad_fn=<MaxBackward1>),
  tensor(0., device='cuda:0', grad_fn=<MinBackward1>)),
 ('lin_node.4.weight',
  tensor(0.0221, device='cuda:0', grad_fn=<MaxBackward1>),
  tensor(-0.0221, device='cuda:0', grad_fn=<MinBackward1>)),
 ('lin_node.4.bias',
  te

In [15]:
output = model(data,True,False,logLoss=logLoss,weight=weight)

In [16]:
output

(tensor(nan, device='cuda:0', grad_fn=<AddBackward0>),
 tensor([4.5542, 3.8246, 1.0540, 2.3506, 1.2550, 1.3202, 1.6085, 0.1599],
        device='cuda:0', grad_fn=<IndexPutBackward>))

In [17]:
self = model

In [18]:
IsTrain=True
typeTrain=False

In [45]:
        out = self.lin_node(data.x)
        # edge_*3 only does not repeat for undirected graph. Hence need to add (j,i) to (i,j) in edges
        edge_index3 = torch.cat([data.edge_index3,data.edge_index3[[1,0]]],1)
        n = data.edge_attr3.shape[0]
        temp_ = self.edge2(torch.cat([data.edge_attr3,data.edge_attr4],1))
        edge_attr3 = torch.cat([temp_,temp_],0)
        edge_attr = self.edge1(data.edge_attr)
        
        if self.useMax:
            u = torch.cat([scatter_max(out, data.batch, dim=0)[0],scatter_max(edge_attr, data.batch[data.edge_index[0]], dim=0)[0]], dim=1)
        else:
            u = torch.cat([scatter_mean(out, data.batch, dim=0),scatter_mean(edge_attr, data.batch[data.edge_index[0]], dim=0)], dim=1)
        u = self.u_mlp(u)
        


In [46]:
            for conv in self.conv1:
                out,edge_attr,u = conv(out, data.edge_index, edge_attr, u, data.batch)

In [54]:
torch.isnan(out).sum()

tensor(0, device='cuda:0')

In [52]:
out.shape,edge_index3.shape,edge_attr3.shape,u.shape,data.batch.shape

(torch.Size([576, 512]),
 torch.Size([2, 3430]),
 torch.Size([3430, 512]),
 torch.Size([32, 512]),
 torch.Size([576]))

In [55]:
x_new, edge_attr_new, u_new = self.conv2[0](out,edge_index3,edge_attr3, u, data.batch)

In [86]:
torch.isnan(out).sum()

tensor(0, device='cuda:0')

In [63]:
x, edge_index, edge_attr, u, batch = out,edge_index3,edge_attr3, u, data.batch

In [67]:
self = self.conv2[0].conv

In [64]:
row, col = edge_index

In [68]:
        if self.edge_model is not None:
            edge_attr = self.edge_model(x[row], x[col], edge_attr, u,
                                        batch if batch is None else batch[row])

In [82]:
x_new = self(x, edge_index, edge_attr, u, batch)

In [74]:
self.node_model

NodeModel(
  (_mlp): Sequential(
    (0): Linear(in_features=1536, out_features=6144, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=6144, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
  )
)

In [78]:
self = self.node_model

In [79]:
self

NodeModel(
  (_mlp): Sequential(
    (0): Linear(in_features=1536, out_features=6144, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=6144, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
  )
)

In [84]:
self.useMax

True

In [156]:
out,_ = scatter_max(edge_attr, edge_index[0], dim=0, dim_size=x.size(0))
#out = scatter_mean(edge_attr, edge_index[0], dim=0, dim_size=x.size(0))

In [158]:
edge_attr.abs().max()

tensor(0.6951, device='cuda:0', grad_fn=<MaxBackward1>)

In [160]:
edge_attr.shape

torch.Size([3430, 512])

In [157]:
out.abs().max()

tensor(3.4028e+38, device='cuda:0', grad_fn=<MaxBackward1>)

In [136]:
edge_index[0].max()

tensor(575, device='cuda:0')

In [151]:
import torch_scatter 

In [152]:
torch_scatter.__version__

'1.3.1'

In [159]:
scatter_max??

In [133]:
edge_index[0]

tensor([  9,   9,   9,  ..., 563, 564, 565], device='cuda:0')

In [130]:
out.abs().max()

tensor(3.4028e+38, device='cuda:0', grad_fn=<MaxBackward1>)

In [111]:
out = torch.cat([x, out, u[batch]], dim=1)

In [112]:
torch.isnan(out).sum()

tensor(0, device='cuda:0')

In [113]:
torch.isnan(self._mlp(out)).sum()

tensor(24576, device='cuda:0')

In [ ]:
torch.isnan(self).sum()

In [118]:
self._mlp

Sequential(
  (0): Linear(in_features=1536, out_features=6144, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=6144, out_features=512, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
)

In [128]:
edge_attr.abs().max()

tensor(0.6951, device='cuda:0', grad_fn=<MaxBackward1>)

In [114]:
out2 = self._mlp[0](out)

In [115]:
torch.isnan(out2).sum()

tensor(0, device='cuda:0')

In [123]:
out2.abs().max()

tensor(inf, device='cuda:0', grad_fn=<MaxBackward1>)

In [120]:
out3 = self._mlp[1](out2)

In [121]:
torch.isnan(out3).sum()

tensor(0, device='cuda:0')

In [122]:
out3.abs().max()

tensor(inf, device='cuda:0', grad_fn=<MaxBackward1>)

In [42]:
conv = 
out,edge_attr3,u = conv(out,edge_index3,edge_attr3, u, data.batch) 

In [43]:
torch.isnan(out).sum()

tensor(24576, device='cuda:0')

In [44]:
conv

MetaLayer_block

In [ ]:
        if self.interleave:
            for conv1,conv2 in zip(self.conv1,self.conv2):
                out,edge_attr,u = conv1(out, data.edge_index, edge_attr, u, data.batch)
                out,edge_attr3,u = conv2(out,edge_index3,edge_attr3, u, data.batch)
        else:
            for conv in self.conv1:
                out,edge_attr,u = conv(out, data.edge_index, edge_attr, u, data.batch)
            for conv in self.conv2:
                out,edge_attr3,u = conv(out,edge_index3,edge_attr3, u, data.batch) 
        
        edge_attr3 = edge_attr3[:n]
        if typeTrain:
            if IsTrain:
                y = data.y[data.type_attr]
            edge_attr3 = edge_attr3[data.type_attr]
            edge_index3 = data.edge_index3[:,data.type_attr]
            edge_attr3_old = data.edge_attr3[data.type_attr]
        else:
            if IsTrain:
                y = data.y
            edge_index3 = data.edge_index3
            edge_attr3_old = data.edge_attr3
            
        yhat = self.head(out,edge_index3,edge_attr3,edge_attr3_old)

In [21]:
torch.isnan(yhat).sum()

tensor(0, device='cuda:0')

In [22]:
y_atom = self.head_atom(out)

In [23]:
torch.isnan(y_atom).sum()

tensor(5760, device='cuda:0')

In [109]:
[(n,w.abs().min()) for n,w in self._mlp[0].named_parameters()]

[('weight', tensor(3.7253e-09, device='cuda:0', grad_fn=<MinBackward1>)),
 ('bias', tensor(2.6058e-06, device='cuda:0', grad_fn=<MinBackward1>))]

In [27]:
torch.isnan(out).sum()

tensor(24576, device='cuda:0')

In [31]:
torch.isnan(out).sum(1)

tensor([512,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0, 512,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
        512,   0,   0,   0,   0,   0,   0,   0,   0,   0, 512,   0,   0,   0,
          0, 512,   0,   0, 512,   0,   0,   0,   0,   0,   0, 512,   0,   0,
          0,   0, 512,   0,   0,   0,   0,   0,   0,   0,   0,   0, 512,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0, 512,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
        512,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0, 512,   0, 512,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0, 512,   0,   0,   0,   0, 512,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  

<module 'pickle' from '/usr/local/lib/python3.6/pickle.py'>